In [ ]:
#import all the libraries
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import geopandas as gpd
from sklearn.linear_model import LogisticRegression
from sklearn import svm
from sklearn.metrics import confusion_matrix
from sklearn.model_selection import train_test_split
from sklearn import datasets
from sklearn.decomposition import PCA
from sklearn import metrics
from sklearn import preprocessing
import urllib

In [ ]:
# To ignore some warnings
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)
warnings.simplefilter(action='ignore', category=UserWarning)

In [ ]:
#set colormap
plt.rcParams['image.cmap'] = 'RdYlGn'

In [ ]:
!pip install cartopy
import geopandas as gpd
from shapely.geometry import Point
import matplotlib.pyplot as plt
import cartopy.crs as ccrs
import requests

# Replace the URL with the raw GeoJSON URL
geojson_url = "https://raw.githubusercontent.com/johan/world.geo.json/master/countries/USA.geo.json"

# Download the GeoJSON file
response = requests.get(geojson_url)
with open("usa.geojson", "wb") as file:
    file.write(response.content)

# Read the GeoJSON file into a GeoDataFrame
usa_geometry = gpd.read_file("usa.geojson")

# Read your original CSV file into a DataFrame
file_path = "https://tidesandcurrents.noaa.gov/publications/techrpt083.csv"
df = pd.read_csv(file_path, header=15)

# Drop the first 15 rows
df = df.iloc[15:].reset_index(drop=True)

df.head()



,Site,PSMSL ID,Latitude,Longitude,Scenario,Background RSL rate (mm/yr),RSL in 2000 (cm),RSL in 2010 (cm),RSL in 2020 (cm),RSL in 2030 (cm),RSL in 2040 (cm),RSL in 2050 (cm),RSL in 2060 (cm),RSL in 2070 (cm),RSL in 2080 (cm),RSL in 2090 (cm),RSL in 2100 (cm),RSL in 2120 (cm),RSL in 2150 (cm),RSL in 2200 (cm)
0,GMSL,0,NaN,NaN,2.5 - MED,0.00,0,4,11,24,41,63,90,123,161,203,250,357.0,554.0,972.0
1,GMSL,0,NaN,NaN,2.5 - LOW,0.00,0,4,10,21,38,60,87,118,155,195,239,340.0,525.0,920.0
2,GMSL,0,NaN,NaN,2.5 - HIGH,0.00,0,5,12,25,42,65,94,127,164,208,256,365.0,570.0,1010.0
3,SAN FRANCISCO,10,37.81,-122.47,0.3 - MED,-0.07,0,3,6,10,13,17,21,25,28,31,34,40.0,47.0,53.0
4,SAN FRANCISCO,10,37.81,-122.47,0.3 - LOW,-0.16,0,1,3,6,8,10,13,15,17,19,19,23.0,25.0,17.0


In [ ]:
data = df.dropna(subset=['Latitude', 'Longitude'])
data

,Site,PSMSL ID,Latitude,Longitude,Scenario,Background RSL rate (mm/yr),RSL in 2000 (cm),RSL in 2010 (cm),RSL in 2020 (cm),RSL in 2030 (cm),RSL in 2040 (cm),RSL in 2050 (cm),RSL in 2060 (cm),RSL in 2070 (cm),RSL in 2080 (cm),RSL in 2090 (cm),RSL in 2100 (cm),RSL in 2120 (cm),RSL in 2150 (cm),RSL in 2200 (cm)
3,SAN FRANCISCO,10,37.81,-122.47,0.3 - MED,-0.07,0,3,6,10,13,17,21,25,28,31,34,40.0,47.0,53.0
4,SAN FRANCISCO,10,37.81,-122.47,0.3 - LOW,-0.16,0,1,3,6,8,10,13,15,17,19,19,23.0,25.0,17.0
5,SAN FRANCISCO,10,37.81,-122.47,0.3 - HIGH,0.02,0,4,8,13,18,23,28,33,38,43,49,55.0,65.0,83.0
6,SAN FRANCISCO,10,37.81,-122.47,0.5 - MED,-0.07,0,3,8,12,17,22,28,33,38,43,48,56.0,70.0,92.0
7,SAN FRANCISCO,10,37.81,-122.47,0.5 - LOW,-0.16,0,2,5,9,13,17,21,26,31,36,38,46.0,52.0,57.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
35026,grid_-28.5_251.5,1011852515,-28.50,-108.50,2.0 - LOW,-1.41,0,7,12,23,37,56,75,99,126,155,191,252.0,381.0,673.0
35027,grid_-28.5_251.5,1011852515,-28.50,-108.50,2.0 - HIGH,0.11,0,10,18,30,47,66,91,119,152,190,232,316.0,472.0,826.0
35028,grid_-28.5_251.5,1011852515,-28.50,-108.50,2.5 - MED,-0.65,0,8,16,29,46,70,97,129,168,209,256,358.0,542.0,1064.0
35029,grid_-28.5_251.5,1011852515,-28.50,-108.50,2.5 - LOW,-1.41,0,3,8,20,37,61,90,120,154,191,232,313.0,483.0,980.0


In [ ]:


# Group by Latitude and Longitude
grouped_data = data.groupby(['Latitude', 'Longitude'])

# Define a function to calculate mode for 'Site' and 'Scenario', and mean for other columns
def calculate_statistics(group):
    result = pd.Series()
    result['Site'] = group['Site'].mode().iloc[0]
    result['Scenario'] = group['Scenario'].mode().iloc[0]

    # Calculate mean for other columns
    for col in group.columns:
        if col not in ['Latitude', 'Longitude', 'Site', 'Scenario']:
            result[col + '_mean'] = group[col].mean()

    return result

# Calculate Mode and Mean for Each Group
result = grouped_data.apply(calculate_statistics).reset_index()

# Rename columns for clarity
result.columns = ['Latitude', 'Longitude', 'Site', 'Scenario'] + [col + '_mean' for col in data.columns if col not in ['Latitude', 'Longitude', 'Site', 'Scenario']]

result


,Latitude,Longitude,Site,Scenario,PSMSL ID_mean,Background RSL rate (mm/yr)_mean,RSL in 2000 (cm)_mean,RSL in 2010 (cm)_mean,RSL in 2020 (cm)_mean,RSL in 2030 (cm)_mean,RSL in 2040 (cm)_mean,RSL in 2050 (cm)_mean,RSL in 2060 (cm)_mean,RSL in 2070 (cm)_mean,RSL in 2080 (cm)_mean,RSL in 2090 (cm)_mean,RSL in 2100 (cm)_mean,RSL in 2120 (cm)_mean,RSL in 2150 (cm)_mean,RSL in 2200 (cm)_mean
0,-28.5,-110.5,grid_-28.5_249.5,0.3 - HIGH,1.011852e+09,-0.65,0.0,5.333333,10.333333,18.166667,28.166667,40.833333,55.111111,71.833333,90.777778,111.277778,134.722222,179.500000,265.277778,464.000000
1,-28.5,-109.5,grid_-28.5_250.5,0.3 - HIGH,1.011853e+09,-0.67,0.0,5.166667,10.333333,18.277778,28.222222,40.555556,55.222222,71.555556,90.666667,111.444444,134.611111,179.277778,264.666667,463.277778
2,-28.5,-108.5,grid_-28.5_251.5,0.3 - HIGH,1.011853e+09,-0.65,0.0,5.222222,10.388889,18.277778,28.111111,41.000000,55.500000,71.888889,90.944444,111.388889,134.611111,178.722222,264.666667,463.000000
3,-27.5,-111.5,grid_-27.5_248.5,0.3 - HIGH,1.011752e+09,-0.62,0.0,5.333333,10.666667,18.666667,29.055556,41.222222,55.722222,72.444444,91.555556,112.277778,135.833333,180.555556,267.166667,467.777778
4,-27.5,-110.5,grid_-27.5_249.5,0.3 - HIGH,1.011752e+09,-0.69,0.0,5.222222,10.666667,18.333333,28.777778,41.055556,55.611111,72.055556,91.166667,112.166667,135.166667,179.666667,265.611111,466.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1941,71.5,-147.5,grid_71.5_212.5,0.3 - HIGH,1.001852e+09,1.64,0.0,7.388889,15.388889,26.277778,38.055556,52.111111,68.000000,86.666667,106.722222,128.333333,150.500000,200.055556,293.277778,478.555556
1942,71.5,-146.5,grid_71.5_213.5,0.3 - HIGH,1.001852e+09,1.68,0.0,7.833333,16.277778,26.777778,38.611111,52.944444,68.833333,87.611111,107.388889,128.944444,151.444444,200.055556,293.500000,480.111111
1943,71.5,-145.5,grid_71.5_214.5,0.3 - HIGH,1.001852e+09,1.70,0.0,8.055556,16.722222,27.944444,40.111111,54.555556,70.333333,89.444444,109.500000,131.000000,152.944444,202.277778,296.000000,481.388889
1944,71.5,-144.5,grid_71.5_215.5,0.3 - HIGH,1.001852e+09,1.73,0.0,7.722222,16.611111,27.611111,40.388889,54.944444,70.444444,89.222222,109.555556,130.833333,152.000000,200.944444,294.111111,480.333333


In [ ]:
unique_sites = df['Site'].unique()

# Print all unique site values
print("Unique Site Values:")
for site in unique_sites:
    print(site)



Unique Site Values:
GMSL
SAN FRANCISCO
NEW YORK
HALIFAX
FERNANDINA BEACH
TROIS-RIVIERES
SEATTLE
PHILADELPHIA
PORT-SAINT-FRANCOIS
POINTE-AU-PERE
BATISCAN
BALTIMORE
HONOLULU
SAN DIEGO
GALVESTON II
TOFINO
VICTORIA
PRINCE RUPERT
CRISTOBAL
QUEBEC
VANCOUVER
HARRINGTON HBR
ATLANTIC CITY
PORTLAND 
KEY WEST
NEUVILLE
POINT ATKINSON
SAINT JOHN
DESCHAILLONS
LEWES
KETCHIKAN
CHARLESTON I
BOSTON
LOS ANGELES
PENSACOLA
LA JOLLA
ASTORIA
SEWARD
DAYTONA BEACH
SEAVEY ISLAND
SEWELLS POINT
HILO
ANNAPOLIS
MAYPORT
EASTPORT
NEWPORT
WASHINGTON DC
WILLETS POINT
MIAMI BEACH
SANDY HOOK
WOODS HOLE
ST. GEORGES
SANTA MONICA
CRESCENT CITY
FRIDAY HARBOR
NEAH BAY
GRONDINES
PORT AUX BASQUES
ST. JOHN'S
FORT PULASKI
WILMINGTON
PORTSMOUTH
JUNEAU
SOLOMON'S ISLAND
GUANTANAMO BAY
SITKA
CHARLOTTETOWN
CEDAR KEY II
NEW LONDON
PROVIDENCE
ALAMEDA
EUGENE ISLAND
YAKUTAT
CHURCHILL
RICHMOND
ADAK  SWEEPER COVE
MASSACRE BAY
SKAGWAY
PORT ISABEL
PORT SAN LUIS
KWAJALEIN
MONTAUK
ST. PETERSBURG
KAHULUI HARBOR
MIDWAY ISLAND
BAR HARBOR
GRAND ISL

In [ ]:
import pandas as pd
import geopandas as gpd
import matplotlib.pyplot as plt
from shapely.geometry import Point
import cartopy.crs as ccrs
import cartopy.feature as cfeature
import seaborn as sns

# Sample DataFrame, replace this with your actual DataFrame
# df = ...

# Create a GeoDataFrame from the DataFrame
geometry = [Point(xy) for xy in zip(df.Longitude, df.Latitude)]
geo_df = gpd.GeoDataFrame(df, geometry=geometry, crs="EPSG:4326")

# Load the Natural Earth dataset for the USA boundaries
usa_boundaries = gpd.read_file(gpd.datasets.get_path('naturalearth_lowres'))
usa_boundaries = usa_boundaries[usa_boundaries['name'] == 'United States']

# Create a color map for different scenarios
unique_scenarios = df['Scenario'].unique()
colors = sns.color_palette("viridis", len(unique_scenarios))

# Ensure we have enough colors for all unique scenarios
scenario_colors = dict(zip(unique_scenarios, colors[:len(unique_scenarios)]))

# Plotting the points on the map
fig, ax = plt.subplots(figsize=(10, 8), subplot_kw={'projection': ccrs.PlateCarree()})
ax.set_extent([-130, -65, 24, 50])  # Adjusted extent values for the USA

# Plot the USA boundaries
ax.add_feature(cfeature.BORDERS, linestyle=':')
ax.add_feature(cfeature.COASTLINE)

# Scatter plot with color-coded scenarios for each site
for site, scenario, lat, lon in df[['Site', 'Scenario', 'Latitude', 'Longitude']].values:
    color = scenario_colors.get(scenario, 'gray')  # Default to gray if scenario not found in the color map
    ax.scatter(lon, lat, s=50, color=color, label=scenario)

# Adding labels and title
ax.set_xlabel('Longitude')
ax.set_ylabel('Latitude')
ax.set_title('Scenario per Site')

# Create a custom legend outside the plot
legend_elements = [plt.Line2D([0], [0], marker='o', color='w', label=label,
                              markerfacecolor=color, markersize=10) for label, color in scenario_colors.items()]
ax.legend(handles=legend_elements, loc='upper left', title='Scenario')

# Show the map
plt.show

<function matplotlib.pyplot.show(close=None, block=None)>

/usr/local/lib/python3.10/dist-packages/cartopy/io/__init__.py:241: DownloadWarning: Downloading: https://naturalearth.s3.amazonaws.com/50m_cultural/ne_50m_admin_0_boundary_lines_land.zip
  warnings.warn(f'Downloading: {url}', DownloadWarning)
/usr/local/lib/python3.10/dist-packages/cartopy/io/__init__.py:241: DownloadWarning: Downloading: https://naturalearth.s3.amazonaws.com/50m_physical/ne_50m_coastline.zip
  warnings.warn(f'Downloading: {url}', DownloadWarning)


Error in callback <function flush_figures at 0x78d948dc92d0> (for post_execute):


KeyboardInterrupt: ignored

In [ ]:
# Create a GeoDataFrame from your DataFrame
geometry = [Point(xy) for xy in zip(df['Longitude'], df['Latitude'])]
geo_df = gpd.GeoDataFrame(df, geometry=geometry, crs="EPSG:4326")

# Perform a spatial join
df_within_usa = gpd.sjoin(geo_df, usa_geometry, op='within')

# Plotting the points on the GeoJSON basemap
fig, ax = plt.subplots(figsize=(10, 8), subplot_kw={'projection': ccrs.PlateCarree()})
ax.set_extent([-130, -65, 24, 50])  # Adjusted extent values for the USA

# Plotting the USA GeoDataFrame as the basemap
usa_geometry.plot(ax=ax, color='lightgray', edgecolor='black', alpha=0.7)

# Plotting each point within the USA on top of the basemap
ax.scatter(df_within_usa['Longitude'], df_within_usa['Latitude'], s=10, color='red', label='Tide Stations')

# Adding labels and title
ax.set_xlabel('Longitude')
ax.set_ylabel('Latitude')
ax.set_title('Tide Stations within the USA')

# Add gridlines
ax.gridlines(draw_labels=True, linestyle='--', alpha=0.5)

# Add legend
ax.legend()

# Show the map
plt.show()

In [ ]:
file_path1 = "https://tidesandcurrents.noaa.gov/publications/techrpt86_PaP_of_HTFlooding.csv"

# Read the CSV file into a DataFrame, considering the 13th row as header
df1 = pd.read_csv(file_path1, header=10)

df1 = df1.iloc[10:].reset_index(drop=True)

df1 = df1.dropna(subset=['Station Name']).reset_index(drop=True)
df1 = df1.rename(columns={df1.columns[0]: 'FLOOD STATUS'})
df1 = df1.rename(columns={df1.columns[1]: 'YEAR'})

df1.head()



In [ ]:
yearly_categories = df1.groupby(['FLOOD STATUS']).max()

yearly_categories.head(10)

In [ ]:
transposed_df1 = df1.transpose()

# Alternatively, you can use the .T attribute
# transposed_df1 = df1.T

transposed_df1.head()

In [ ]:
from google.colab import files

uploaded = files.upload()

In [ ]:
import zipfile
import io

# Replace 'your_uploaded_file.zip' with the actual name of your uploaded zip file
zip_file_name = 'nmdb-new-mortgage-statistics-all-annual.zip'

with zipfile.ZipFile(io.BytesIO(uploaded[zip_file_name]), 'r') as zip_ref:
    zip_ref.extractall('/content/')

In [ ]:
import pandas as pd

# Replace 'your_csv_file.csv' with the actual name of your CSV file
csv_file_name = 'nmdb-new-mortgage-statistics-all-annual.csv'

df2 = pd.read_csv(f'/content/{csv_file_name}')
df2.head(100)

In [ ]:
df2.columns

In [ ]:
df2['GEOID'].unique()

In [ ]:
from google.colab import files

uploaded = files.upload()

In [ ]:
# Replace 'your_csv_file.csv' with the actual name of your CSV file
csv_file_name1 = '2022_PUDB_EXPORT_123122.csv'

df3= pd.read_csv(f'/content/{csv_file_name1}')
df3.head(100)

In [ ]:
df3.columns

In [ ]:
df3['PropertyType'].unique()

In [ ]:
from google.colab import files

uploaded = files.upload()